In [2]:
# Construct a realistic Rasa 3.10.8-style sample payload with ~100 slots and flow data,
# then save it as JSON for download.

import json, random, time, uuid
from datetime import datetime, timedelta, timezone
from pathlib import Path

now = datetime.now(timezone.utc)
def ts(dt): 
    return dt.isoformat()

# Start with a seed list of realistic slot names often seen in enterprise assistants
seed_slots = [
    "confirm_correction", "accident_grade", "accident_grade_told", "application", "application_told",
    "bom_level", "bom_level_told", "bonus_yn", "case_id", "category", "category_told",
    "component", "component_told", "customer_id", "customer_name", "deadline_date", "deadline_date_told",
    "department", "device_model", "device_os", "email", "employee_id", "escalation_level",
    "file_attachment", "incident_date", "incident_location", "incident_summary", "incident_type",
    "language", "location", "manager_email", "manager_name", "order_id", "order_status",
    "phone_number", "priority", "product_code", "product_name", "project_code", "project_name",
    "quantity", "reason", "region", "request_id", "request_type", "resolution", "severity",
    "shipment_tracking", "site", "site_told", "status", "subject", "ticket_id", "ticket_url",
    "user_confirmed", "user_intent", "user_language", "user_name", "user_timezone", "verification_code",
    "version", "workflow_id", "attachment_urls", "followup_required", "followup_date",
    "sla_hours", "sla_breached", "approval_needed", "approver", "approver_email", "approver_status",
    "cost_center", "budget_code", "risk_level", "due_date", "start_date", "end_date", "env",
    "region_code", "feature_flag", "model_name", "model_version", "gpu_required", "cpu_required",
    "memory_required", "namespace", "k8s_service", "k8s_node", "ingress_needed", "api_endpoint",
    "auth_type", "rbac_role", "owner", "auditor", "tags", "notes", "meta_json"
]

# If fewer than 100, add extra_n to reach 100
target_count = 100
slots_names = seed_slots[:]
i = 1
while len(slots_names) < target_count:
    slots_names.append(f"extra_{i:02d}")
    i += 1

assert len(slots_names) == target_count

# Build slot values with diversity
def fake_value(name):
    if name.endswith("_told"):
        return random.choice([True, False, "False", "True"])
    if "date" in name:
        return (now + timedelta(days=random.randint(-10, 30))).date().isoformat()
    if name in {"confirm_correction", "resolution"}:
        return None
    if name in {"accident_grade", "application", "component", "attachment_urls", "tags"}:
        return []
    if "email" in name:
        return "alex.lee@example.com"
    if name in {"priority", "severity"}:
        return random.choice(["low", "medium", "high", "critical"])
    if name in {"user_confirmed", "approval_needed", "followup_required", "sla_breached", "ingress_needed"}:
        return random.choice([True, False])
    if "id" in name or name.endswith("_code") or name in {"ticket_id"}:
        return str(uuid.uuid4())[:8]
    if name in {"status", "order_status", "approver_status"}:
        return random.choice(["new", "in_progress", "waiting_approval", "done", "blocked"])
    if name in {"workflow_id"}:
        return str(uuid.uuid4())
    if name in {"gpu_required", "cpu_required", "memory_required"}:
        return random.choice([1, 2, 4, 8, 16, "8Gi", "16Gi", "32Gi"])
    if name in {"model_version", "version"}:
        return random.choice(["v1", "v1.1", "v2.0", "2025.08"])
    if name in {"region", "region_code"}:
        return random.choice(["APAC", "EMEA", "NA", "KR-SEO"])
    if name in {"env"}:
        return random.choice(["dev", "staging", "prod"])
    if name in {"site"}:
        return random.choice(["A1", "A2", "A3", "A4"])
    if name in {"reason"}:
        return "Routine check"
    if name in {"notes"}:
        return "User requested expedited handling."
    if name in {"meta_json"}:
        return {"source":"api","confidence": round(random.uniform(0.6, 0.99), 2)}
    return ""

slots_obj = {name: fake_value(name) for name in slots_names}

# Simulate flows (Rasa Pro flows-style)
start_time = now - timedelta(minutes=2)
mid_time = now - timedelta(minutes=1)
flows = [
    {
        "name": "report_incident",
        "status": "completed",
        "required_slots": ["incident_type", "incident_date", "incident_location", "incident_summary", "severity"],
        "filled_slots": {
            "incident_type": "quality_issue",
            "incident_date": (now - timedelta(days=1)).date().isoformat(),
            "incident_location": "FAB-A1",
            "incident_summary": "Anomaly detected on line 3; OLED thickness out of spec.",
            "severity": "high"
        },
        "start_time": ts(start_time),
        "end_time": ts(mid_time),
        "steps": [
            {"type": "user", "text": "I need to report an issue on line 3."},
            {"type": "bot", "action": "utter_ask_incident_type"},
            {"type": "user", "text": "quality issue"},
            {"type": "bot", "action": "utter_ask_incident_date"},
            {"type": "user", "text": (now - timedelta(days=1)).date().isoformat()},
            {"type": "bot", "action": "utter_confirm_submit"},
            {"type": "bot", "action": "action_create_ticket", "result": {"ticket_id": "TCK-48291"}}
        ]
    },
    {
        "name": "submit_application",
        "status": "in_progress",
        "required_slots": ["application", "approver", "due_date"],
        "filled_slots": {
            "application": ["capacity_increase"],
            "approver": "Minji Park",
            "due_date": (now + timedelta(days=3)).date().isoformat()
        },
        "start_time": ts(mid_time),
        "end_time": None,
        "steps": [
            {"type": "bot", "action": "utter_ask_application"},
            {"type": "user", "text": "capacity increase"},
            {"type": "bot", "action": "utter_ask_approver"},
            {"type": "user", "text": "Minji Park"},
            {"type": "bot", "action": "utter_ask_due_date"},
            {"type": "user", "text": (now + timedelta(days=3)).date().isoformat()}
        ]
    }
]

payload = {
    "messages": [
        {"recipient_id": "opus", "text": "sdfsdf"},
        {"recipient_id": "opus", "text": "Got it. Creating your ticket now."},
        {"recipient_id": "opus", "text": "Ticket created: TCK-48291. Do you want to submit an application for capacity increase?"}
    ],
    "metadata": {
        "x-genos-workflow-id": None,
        "sender_ip": "10.0.0.25",
        "channel": "rest",
        "received_at": ts(now)
    },
    "conversation_id": "opus",
    "current_flow": "submit_application",
    "processed_flows": ["report_incident"],
    "flows": flows,
    "slots": slots_obj,
    "tracker": {
        "sender_id": "opus",
        "latest_event_time": ts(now),
        "events": [
            {"event": "user", "timestamp": ts(start_time), "text": "sdfsdf"},
            {"event": "action", "timestamp": ts(start_time + timedelta(seconds=5)), "name": "action_listen"},
            {"event": "bot", "timestamp": ts(mid_time), "metadata": {"action_name":"action_create_ticket"}, "text": "Ticket created: TCK-48291"}
        ]
    }
}

# Write to file
out_path = Path("data/sample_rasa_payload.json")
out_path.write_text(json.dumps(payload, ensure_ascii=False, indent=2))
print(f"Saved to: {out_path}")
print("Slot count:", len(slots_obj))
print("Flow names:", [f['name'] for f in flows])


Saved to: data\sample_rasa_payload.json
Slot count: 100
Flow names: ['report_incident', 'submit_application']
